# Neptune as Graph Memory

In this notebook we will be showing you how Mem0 stores Memories as vectors in OpenSearch, and as graph relations in Neptune Analytics.

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Connect to Neptune

To connect to AWS Neptune Analytics, you need to configure Neptune with your AWS profile details. I have configured AWS credentials in my home profile and can connect by specifying the AWS_PROFILE environment variable

TODO: Reference to: Neptune setup; aws-langchain

### 3. Configure OpenSearch

We're going to use OpenSearch as our vector store.  You can run [OpenSearch from docker image](https://docs.opensearch.org/docs/latest/install-and-configure/install-opensearch/docker/):

```bash
docker pull opensearchproject/opensearch:3
```

And verify that it's running with a `<custom-admin-password>`:

```bash
 docker run -d -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=<custom-admin-password>" opensearchproject/opensearch:latest

 curl https://localhost:9200 -ku admin:<custom-admin-password>
```

We're going to connect [OpenSearch using the python client](https://github.com/opensearch-project/opensearch-py):

```bash
pip install "opensearch-py"
```

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [1]:
from mem0 import Memory
from opensearchpy import OpenSearch
from langchain_aws import NeptuneAnalyticsGraph
import os, logging, sys

logging.getLogger('mem0.memory.neptune_memory').setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logging.basicConfig(
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout  # Explicitly set output to stdout
)

Setup the Mem0 configuration using:
- openai as the embedder
- AWS Neptune Analytics instance as a graph store
- OpenSearch as the vector store

In [2]:
graph_identifier = os.environ.get("GRAPH_ID")
opensearch_username = os.environ.get("OS_USERNAME")
opensearch_password = os.environ.get("OS_PASSWORD")
config = {
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large",
            "embedding_dims": 1536
        },
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "graph_identifier": graph_identifier,
            # "endpoint_url": endpoint_url,
            # "region": region,
        },
    },
    "vector_store": {
        "provider": "opensearch",
        "config": {
            "collection_name": "vector_store",
            "host": "localhost",
            "port": 9200,
            "user": opensearch_username,
            "password": opensearch_password,
            "use_ssl": False,
            "verify_certs": False,
        },
    },
}

## Graph Memory initializiation 

Initialize Memgraph as a Graph Memory store: 

In [3]:
m = Memory.from_config(config_dict=config)

app_id = "indian_questions"
user_id = "user1"

/Users/andrewc/git/acarbonetto/mem0/.venv/lib/python3.12/site-packages/botocore/auth.py:425: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()


## Store memories 

Create memories:

In [4]:
# First turn: Initial conversation about Indian famous people
messages1=[{"role": "user", "content": "user wants to know about famous people from india"},
           {"role": "assistant", "content": "India has produced many famous individuals across various fields. Here are a few notable figures: \
    Mahatma Gandhi - Known for his role in India's independence movement through nonviolent civil disobedience. \
    Dr. B.R. Ambedkar - An influential social reformer and the principal architect of the Indian Constituti2on. \
    Rabindranath Tagore - A poet, writer, and the first non-European to win the Nobel Prize in Literature. \
    Mother Teresa - Renowned for her humanitarian work and dedication to helping the poor and sick. A.P.J. Abdul Kalam - A scientist and the 11th President of India, known for his work in aerospace engineering and his inspirational leadership. \
    These individuals have made significant contributions to society and have left a lasting impact both in India and around the world. "}]
result1=m.add(
    messages1,
    metadata={"app_id": app_id},
    user_id="user1"
)

print("----Turn 1 Memory Result:----")
for n in result1['results']:
    print(f"node id={n['id']} memory={n['memory']}")
for e in result1['relations']['added_entities']:
    print(f"edge \"{e[0]["source"]}\" --{e[0]["relationship"]}--> \"{e[0]["target"]}\"")



DEBUG - Entity type map: {'india': 'location', 'mahatma_gandhi': 'person', 'dr._b.r._ambedkar': 'person', 'rabindranath_tagore': 'person', 'mother_teresa': 'person', 'a.p.j._abdul_kalam': 'person'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'India', 'entity_type': 'Location'}, {'entity': 'Mahatma Gandhi', 'entity_type': 'Person'}, {'entity': 'Dr. B.R. Ambedkar', 'entity_type': 'Person'}, {'entity': 'Rabindranath Tagore', 'entity_type': 'Person'}, {'entity': 'Mother Teresa', 'entity_type': 'Person'}, {'entity': 'A.P.J. Abdul Kalam', 'entity_type': 'Person'}]}}]}
DEBUG - Extracted entities: [{'source': 'mahatma_gandhi', 'relationship': 'known_for', 'destination': "role_in_india's_independence_movement_through_nonviolent_civil_disobedience"}, {'source': 'dr._b.r._ambedkar', 'relationship': 'known_for', 'destination': 'influential_social_reformer_and_the_principal_architect_of_the_indian_constitution'}, {'source': 'ra

In [5]:
# Second turn: User asks about Indian cuisine
messages2=[{"role": "user", "content": "What are some famous Indian dishes I should try?"},
           {"role": "assistant", "content": "India offers a rich variety of delicious dishes. Here are some you should definitely try:\
    \n\n1. Butter Chicken - A creamy tomato-based curry with tender chicken pieces\
    \n2. Masala Dosa - A crispy rice pancake filled with spiced potatoes\
    \n3. Biryani - Fragrant rice dish with meat or vegetables and aromatic spices\
    \n4. Samosas - Triangular pastries filled with spiced potatoes and peas\
    \n5. Chaat - A category of savory snacks featuring crispy elements, yogurt, and tangy chutneys\
    \n6. Palak Paneer - Cottage cheese cubes in a spinach-based sauce\
    \n\nThese dishes represent different regions of India and showcase the diverse flavors of Indian cuisine."}]
result2=m.add(
    messages2,
    metadata={"app_id": app_id},
    user_id="user1"
)

print("----Turn 2 Memory Result:----")
for n in result2['results']:
    print(f"node id={n['id']} memory={n['memory']}")
for e in result2['relations']['added_entities']:
    print(f"edge \"{e[0]["source"]}\" --{e[0]["relationship"]}--> \"{e[0]["target"]}\"")


DEBUG - Entity type map: {'india': 'location', 'butter_chicken': 'dish', 'masala_dosa': 'dish', 'biryani': 'dish', 'samosas': 'dish', 'chaat': 'dish', 'palak_paneer': 'dish'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'India', 'entity_type': 'Location'}, {'entity': 'Butter Chicken', 'entity_type': 'Dish'}, {'entity': 'Masala Dosa', 'entity_type': 'Dish'}, {'entity': 'Biryani', 'entity_type': 'Dish'}, {'entity': 'Samosas', 'entity_type': 'Dish'}, {'entity': 'Chaat', 'entity_type': 'Dish'}, {'entity': 'Palak Paneer', 'entity_type': 'Dish'}]}}]}
DEBUG - Extracted entities: [{'source': 'india', 'relationship': 'offers', 'destination': 'butter_chicken'}, {'source': 'india', 'relationship': 'offers', 'destination': 'masala_dosa'}, {'source': 'india', 'relationship': 'offers', 'destination': 'biryani'}, {'source': 'india', 'relationship': 'offers', 'destination': 'samosas'}, {'source': 'india', 'relationship': 'offers', 

Create Agent Memories

In [6]:
# Third turn: Assistant recognizes user's food preferences
messages3=[{"role": "assistant", "content": "That's great to hear you enjoy spicy food, particularly vindaloo! Vindaloo is indeed known for its fiery heat and complex flavors, originally from Goa but popular throughout India and abroad. Your experience trying it in Mumbai during your business trip last year must have been authentic. Mumbai has some excellent restaurants serving regional cuisines from across India. Did you get to try any other local specialties during your visit? Many travelers find that the food in India varies dramatically from region to region."}]
result3=m.add(
    messages3,
    metadata={"app_id": app_id},
    agent_id="agent1"
)

print("----Turn 3 Memory Result:----")
for n in result3['results']:
    print(f"node id={n['id']} memory={n['memory']}")
for e in result3['relations']['added_entities']:
    print(f"edge \"{e[0]["source"]}\" --{e[0]["relationship"]}--> \"{e[0]["target"]}\"")


DEBUG - Entity type map: {'vindaloo': 'dish', 'goa': 'location', 'india': 'location', 'mumbai': 'location', 'business_trip': 'event', 'last_year': 'time'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'vindaloo', 'entity_type': 'dish'}, {'entity': 'Goa', 'entity_type': 'location'}, {'entity': 'India', 'entity_type': 'location'}, {'entity': 'Mumbai', 'entity_type': 'location'}, {'entity': 'business trip', 'entity_type': 'event'}, {'entity': 'last year', 'entity_type': 'time'}]}}]}
DEBUG - Extracted entities: [{'source': 'vindaloo', 'relationship': 'originates_from', 'destination': 'goa'}, {'source': 'vindaloo', 'relationship': 'popular_in', 'destination': 'india'}, {'source': 'business_trip', 'relationship': 'occurred_in', 'destination': 'last_year'}, {'source': 'business_trip', 'relationship': 'took_place_in', 'destination': 'mumbai'}, {'source': 'mumbai', 'relationship': 'serves', 'destination': 'regional_cuisines'}

Add run-specific memories

In [7]:
result4 = m.add(
    messages3,
    metadata={"app_id": app_id},
    run_id="trip-planning-2024"
)
print("trip-planning-2024 Result:", result4)

DEBUG - Entity type map: {'vindaloo': 'dish', 'goa': 'location', 'india': 'location', 'mumbai': 'location', 'last_year': 'time', 'business_trip': 'event', 'regional_cuisines': 'cuisine'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'vindaloo', 'entity_type': 'dish'}, {'entity': 'Goa', 'entity_type': 'location'}, {'entity': 'India', 'entity_type': 'location'}, {'entity': 'Mumbai', 'entity_type': 'location'}, {'entity': 'last year', 'entity_type': 'time'}, {'entity': 'business trip', 'entity_type': 'event'}, {'entity': 'regional cuisines', 'entity_type': 'cuisine'}]}}]}
DEBUG - Extracted entities: [{'source': 'vindaloo', 'relationship': 'is_originated_from', 'destination': 'goa'}, {'source': 'vindaloo', 'relationship': 'is_popular_in', 'destination': 'india'}, {'source': 'vindaloo', 'relationship': 'is_served_in', 'destination': 'mumbai'}, {'source': 'mumbai', 'relationship': 'is_located_in', 'destination': 'india'}, 

## Search memories

In [8]:
for result in m.search("What do we know about this user?", user_id=user_id, limit=10)["results"]:
    print(f"\"{result["memory"]}\" [score: {result["score"]}]")

DEBUG - Entity type map: {'user': 'person'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'user', 'entity_type': 'person'}]}}]}
DEBUG - _search_graph_db
  query=
            MATCH (n)
            WHERE n.embedding IS NOT NULL AND n.user_id = $user_id
            WITH n, n.embedding AS embedding, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, round(2 * distance - 1) AS similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION
                MATCH (m)-[r]->(n) 
                RETURN m.name AS 

Neptune Analytics Graph

In [ ]:
all_results = m.get_all(user_id=user_id)
for n in all_results['results']:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results['relations']:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

In [ ]:
### NOTE: search/get_all by agent_id or run_id is not yet supported

all_results = m.get_all(agent_id="agent1")
for n in all_results['results']:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results['relations']:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

In [ ]:
neptune_graph = NeptuneAnalyticsGraph(graph_identifier)

query = """
        MATCH (n {user_id: $user_id})-[r]->(m {user_id: $user_id})
        RETURN n.name AS source, type(r) AS relationship, m.name AS target
        LIMIT $limit
        """
edge_results = neptune_graph.query(
    query, params={"user_id": user_id, "limit": 100}
)
print("----RELATIONSHIPS----")
for e in edge_results:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")


In [ ]:
opensearch_index = "vector_store"
vector_store_config = config[opensearch_index]["config"]

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': vector_store_config["host"], 'port': vector_store_config["port"]}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (vector_store_config["user"], vector_store_config["password"]),
    use_ssl = vector_store_config["use_ssl"],
    verify_certs = vector_store_config["verify_certs"],
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

query = {"query":{"match_all": {}}}

response = client.search(
    body = query,
    index = opensearch_index,
)

print("----VECTORS----")
for v in response["hits"]["hits"]:
    print(f"vector id=\"{v["_source"]["id"]}\" data={v["_source"]["payload"]["data"]}")

Reset memories: vectors, and graph relationships

In [9]:
m.reset()
# only works for neptune_memory
memory_graph = m.graph.reset()

WARNING - Resetting all memories
WARNING - Resetting index vector_store...
WARNING - Clearing graph...
